In [2]:
import numpy as np
import nltk
from nltk.corpus import brown
import math
import re
from gensim import models
from sklearn import metrics
nltk.download("brown")
nltk.download("universal_tagset")
nltk.download("punkt")

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
sentences = brown.tagged_sents(tagset="universal")
word_tag=brown.tagged_words(tagset='universal')
words_tags=np.array(word_tag)
unique_tags=np.unique(words_tags[:,1])
tag_dic ={ }
rev_dic={}
tag_dic["pad"]=0
rev_dic[0]="pad"
for A, B in zip(unique_tags, np.arange(unique_tags.size)):
    tag_dic[A] = B+1
    rev_dic[B+1]=A


In [4]:
def remove_comma(match_obj):
    if match_obj.group(2) is not None:
        return match_obj.group(1)+match_obj.group(3)
def remove_1dash(match_obj):
    if match_obj.group(1) is not None:
        return match_obj.group(2)
def preprocess_word(w):
    w=w.lower()
    w=re.sub("n't","",w)
    w=re.sub("ed$","ing",w)
    w= re.sub(r"([a-zA-Z])(,+)([a-zA-Z])", remove_comma, w)
    w= re.sub(r"([0-9]+)([,.]+)([0-9]+)", remove_comma, w)
    w=re.sub("[0-9]+","1",w)
    w=re.sub("\$.+","$",w)
    w= re.sub(r"(1\-)(.+)", remove_1dash, w)
    return w

In [5]:
# sentences1 = []
# words = []
# for sentence in sentences:
#     sentence1 = []
#     for word in sentence:
#         sentence1.append([preprocess_word(word[0]),word[1]])
#         words.append(preprocess_word(word[0]))
#     sentences1.append(sentence1)

In [6]:
# words = np.unique(np.sort(np.array(words)))

In [7]:
# wordDict = {}
# for i in range(len(words)):
#     wordDict[words[i]] = i

In [8]:
# wordDict

In [9]:
import gensim.downloader as api
from gensim.models import KeyedVectors
# wv = api.load('word2vec-google-news-300')
# wv.save("google_wv.wordvectors")
# vec_king = wv['king']
wv = KeyedVectors.load("google_wv.wordvectors", mmap='r')

In [10]:
# def getVectorFromW2V(str):
#     if str in wv:
#         return wv[str]
#     else:
#         return None
# found=0
# notfound=0
# w2v_embeddings = np.random.normal(0,1,(len(wordDict) + 1, 300))
# for word,i in wordDict.items():
#     w2v_vector = getVectorFromW2V(word)
#     if w2v_vector is not None:
#         found+=1
#         w2v_embeddings[i] = w2v_vector
#     else:
#       notfound+=1



In [11]:
def get_vector_sentence(sentence,n,present=None):
    length=len(sentence)
    no_parts=math.ceil(length/n)
    pads=length-(n*no_parts)
    labels=np.zeros((no_parts,n))
    word_v=np.random.normal(0,1,(no_parts,n,300))
    for i in range(no_parts):
      for j in range(n):
        if n*i+j>=length:
          word_v[i,j,:]=np.zeros(300)
          continue
        if present is not None:
          word=preprocess_word(sentence[n*i+j][0])
          labels[i,j]=tag_dic[sentence[n*i+j][1]]
          if word in wv:
            word_v[i,j,:]=wv[word].squeeze()
        else:
          word=preprocess_word(sentence[n*i+j])
          if word in wv:
            word_v[i,j,:]=wv[word].squeeze()
    labels_t=np.tile(labels[:,:,None],(1,1,13))
    labels_r=np.tile(np.arange(13)[None,None,:],(no_parts,n,1))
    mask=labels_t==labels_r
    labels_out=np.zeros((no_parts,n,13))
    labels_out[mask]=1
    return np.reshape(word_v,(no_parts,300*n)),np.reshape(labels_out,(no_parts,n*13))


In [12]:
n_max=10
# k=0
# sent_vector_dic={}
# sent_label_dic={}
# for sentence in sentences:
#     vect,labs=get_vector_sentence(sentence,n_max,1)
#     for l in range(vect.shape[0]):
#       sent_vector_dic[k]=vect[l]
#       sent_label_dic[k]=labs[n_max*l:n_max*(l+1)]
#       k+=1
#     print(k)

In [13]:
import torch 
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
def split_index(n,val_perc):         #n=size of dataset #val_perc=percentage of validation set from net train dataset.
    n_val=int(val_perc*n)
    index=np.random.permutation(n)
    return index[n_val:],index[:n_val]

In [33]:
batch_size=128
total_sent=len(sentences)
train_index,validation_index=split_index(total_sent,0.2)
no_batches_train=math.ceil(len(train_index)/batch_size)
no_batches_val=math.ceil(len(validation_index)/batch_size)
current_batch_train=0
current_batch_test=0
train_loader=5
val_loader=10


In [16]:
def give_batch_train():
  global current_batch_train
  global current_batch_test
  global no_batches_train
  global no_batches_val
  global train_loader
  global val_loader
  if current_batch_train==no_batches_train-1:
    l=train_index[current_batch_train*batch_size:]
  else:
    l=train_index[current_batch_train*batch_size:(current_batch_train+1)*batch_size]
  mini_size=len(l)
  vect=None
  labs=None
  for i in range(mini_size):
      vects_c,labs_c=get_vector_sentence(sentences[l[i]],n_max,1)
      if vect is not None:
        vect=np.concatenate((vect,vects_c),0)
        labs=np.concatenate((labs,labs_c),0)
      else:
        vect=vects_c.copy()
        labs=labs_c.copy()
  current_batch_train=(current_batch_train+1)%no_batches_train
  return torch.from_numpy(vect),torch.from_numpy(labs)

def give_batch_test():
  global current_batch_train
  global current_batch_test
  global no_batches_train
  global no_batches_val
  global train_loader
  global val_loader
  if current_batch_test==no_batches_val-1:
    l=validation_index[current_batch_test*batch_size:]
  else:
    l=validation_index[current_batch_test*batch_size:(current_batch_test+1)*batch_size]
  mini_size=len(l)
  vect=None
  labs=None
  for i in range(mini_size):
      vects_c,labs_c=get_vector_sentence(sentences[l[i]],n_max,1)
      if vect is not None:
        vect=np.concatenate((vect,vects_c),0)
        labs=np.concatenate((labs,labs_c),0)
      else:
        vect=vects_c.copy()
        labs=labs_c.copy()
  current_batch_test=(current_batch_test+1)%no_batches_val
  return torch.from_numpy(vect),torch.from_numpy(labs)

In [17]:
class my_model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.Linear_1=torch.nn.Linear(n_max*300,n_max*104)
        self.Linear_2=torch.nn.Linear(n_max*104,n_max*52)
        self.Linear_3=torch.nn.Linear(n_max*52,n_max*13)
    def forward(self,input_batch):
        input_batch=torch.reshape(input_batch,(-1,n_max*300))
        output=self.Linear_1(input_batch)
        output=torch.nn.functional.relu(output)
        output=self.Linear_2(output)
        output=torch.nn.functional.relu(output)
        output=self.Linear_3(output)
        output=torch.reshape(output,(-1,n_max,13))
        output=torch.exp(output)
        output2=torch.sum(output,2,True)
        output=torch.div(output,output2)
        output=torch.reshape(output,(-1,n_max*13))
        return output

In [18]:
def loss_function(preds,labels):
    loss=-torch.mean(torch.sum(torch.multiply(labels,torch.log(preds)),1))
    return loss

In [19]:
# input_size=300*n_max
# num_classes=13
model=my_model()
# loss_function=torch.nn.functional.cross_entropy
learning_rate=0.1
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)
train_accuracy=[]
validation_accuracy=[]

In [20]:
def single_batch(model,loss_function,inputbatch,labels,optimizer=None,metric=None):
    preds=model(inputbatch.float())
    loss=loss_function(preds,labels)
    if optimizer is not None:
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    metric_result=None
    if metric is not None:
        metric_result=metric(preds,labels)
    return loss.item(),inputbatch.shape[0],metric_result

In [21]:
def evaluate(model,loss_function,loader,metric=None):
    global current_batch_train
    global current_batch_test
    global no_batches_train
    global no_batches_val
    global train_loader
    global val_loader
    with torch.no_grad():
        results=[]
        if loader==5:
          for i in range(no_batches_train):
            xbatch,ybatch=give_batch_train()
            results.append(single_batch(model,loss_function,xbatch,ybatch,metric=metric))
        else:
          for i in range(no_batches_val):
            xbatch,ybatch=give_batch_test()
            results.append(single_batch(model,loss_function,xbatch,ybatch,metric=metric))
        losses,single_batchsize,metrics=zip(*results)
        dataset_size=np.sum(single_batchsize)
        average_loss=n_max*np.sum(np.multiply(losses,single_batchsize))/dataset_size
        average_metric=None
        if metric is not None:
            average_metric=np.sum(np.multiply(metrics,single_batchsize))/dataset_size
        return average_loss,dataset_size,average_metric

In [22]:
def accuracy(preds,labels):
    global n_max
    preds=torch.reshape(preds,(-1,n_max,13))
    preds=torch.argmax(preds,dim=2)
    labels=torch.reshape(labels,(-1,n_max,13))
    labels=torch.argmax(labels,dim=2)
    return torch.sum(preds==labels).item()/(preds.shape[0]*preds.shape[1])

In [23]:
def plot_graph(train_accuracy,validation_accuracy):
    plt.plot(train_accuracy,color='red',label='train')
    plt.plot(validation_accuracy,color='green',label='validation')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title("Accuracy vs No.of epochs")

In [24]:
def train(epochs,model,loss_function,optimizer,metric=None):
    global current_batch_train
    global current_batch_test
    global no_batches_train
    global no_batches_val
    global train_loader
    global val_loader
    for epoch in range(epochs):
        #train the model:
        for i in range(no_batches_train):
          xbatch,ybatch=give_batch_train()
          loss,_,_=single_batch(model,loss_function,xbatch,ybatch,optimizer=optimizer)
          print(i," ",loss)
        print("Epoch Number:",epoch+1)
        if metric is not None:
            #Performance on training Set:
            # train_loss,tr_dataset_size,tr_average_acc=evaluate(model,loss_function,train_loader,metric)
            # print("Training Set Size:",tr_dataset_size,"  Average loss per example in training set:",train_loss,"  accuracy percent:",tr_average_acc*100,"%")
            #Performance on Validation Set:
            val_loss,v_dataset_size,v_average_acc=evaluate(model,loss_function,val_loader,metric=accuracy)
            print("Validation Set Size:",v_dataset_size,"  Average loss per example in validation set:",val_loss,"  accuracy percent:",v_average_acc*100,"%")
            print()
            # train_accuracy.append(tr_average_acc)
            validation_accuracy.append(v_average_acc)
        else:
            #Performance on training Set:
            train_loss,tr_dataset_size,_=evaluate(model,loss_function,train_loader,metric)
            print("Training Set Size:",tr_dataset_size,"  Average loss per example in training set:",train_loss)
            #Performance on Validation Set:
            val_loss,v_dataset_size,_=evaluate(model,loss_function,val_loader,metric=accuracy)
            print("Validation Set Size:",v_dataset_size,"  Average loss per example in validation set:",val_loss)
            print()

In [ ]:
val_loss,v_dataset_size,v_average_acc=evaluate(model,loss_function,val_loader,metric=accuracy)
print("Validation Set Size:",v_dataset_size,"  Average loss per example in validation set:",val_loss,"  accuracy percent:",v_average_acc*100,"%")
train_loss,t_dataset_size,t_average_acc=evaluate(model,loss_function,train_loader,metric=accuracy)
print("Training Set Size:",t_dataset_size,"  Average loss per example in Training set:",train_loss,"  accuracy percent:",t_average_acc*100,"%")
train_accuracy.append(t_average_acc)
validation_accuracy.append(v_average_acc)

KeyboardInterrupt: ignored

In [30]:
model3=my_model()
model3.load_state_dict(torch.load('third.pth'))
optimizer3=torch.optim.Adam(model3.parameters(),lr=0.00001)

In [31]:
train(1,model3,loss_function,optimizer3,accuracy)

0   2.9561164896889944
1   3.0865809189725604
2   3.207040956448348
3   3.0326874554613115
4   3.0681389129025094
5   3.376988254780948
6   3.0363884848109857
7   3.2256866133733038
8   3.3346203356886517
9   3.161812474670766
10   3.028764956068893
11   3.207746245228713
12   3.023549506029227
13   3.1311860451610767
14   3.1862557063003316
15   3.335261300364937
16   3.19615877561889
17   3.1493487062739964
18   3.115772361794189
19   3.2855827790721333
20   3.259774762583246
21   3.2934399362848463
22   3.1208256319474796
23   3.066909872327912
24   3.123009884149562
25   3.2144359465809016
26   3.2401381123924637
27   3.166072642059624
28   3.2211862542570637
29   3.123276758816472
30   3.3834242340703686
31   3.149151699097861
32   3.1774924702454816
33   3.3114660093537296
34   3.147836797709584
35   3.022010283996375
36   3.2706641277215414
37   3.2365123492918944
38   3.118031842924777
39   3.214378226426343
40   3.3022639543634718
41   3.2548109658523954
42   3.161269501429396

In [36]:
torch.save(model3.state_dict(),'third.pth')

In [35]:
train_loss,t_dataset_size,t_average_acc=evaluate(model3,loss_function,train_loader,metric=accuracy)
print("Training Set Size:",t_dataset_size,"  Average loss per example in Training set:",train_loss,"  accuracy percent:",t_average_acc*100,"%")

Training Set Size: 113784   Average loss per example in Training set: nan   accuracy percent: 89.04222034732476 %
